In [2]:
!pip install langchain==0.0.270
!pip install faiss-cpu langchain==0.0.271 openai tiktoken

  Using cached langchain-0.0.270-py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.0.270-py3-none-any.whl (1.5 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.271
    Uninstalling langchain-0.0.271:
      Successfully uninstalled langchain-0.0.271
  Using cached langchain-0.0.271-py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.0.271-py3-none-any.whl (1.5 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.270
    Uninstalling langchain-0.0.270:
      Successfully uninstalled langchain-0.0.270


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [5]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [6]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1goH4wHT1SDvvDEWo0VwuCmFncKsIfTFb')


In [7]:
data_from_url

'\ufeffОнихолизис\r\n\r\n\r\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа. ><\r\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа. ><\r\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком. ><\r\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия. ><\r\n\r\n\r\n\r\n\r\nПричины появления онихолизиса\r\nВ большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису. ><\r\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случае каждый шаг является травмой для ногтевой пластины. ><\r\nТа

In [8]:
# Открываем файл для чтения
with open('C:/Users/lenovo/Desktop/ITE Group/AI_STAFF/Neuro/BASE.txt', 'r', encoding='utf-8') as file:
    # Читаем содержимое файла и сохраняем его в переменную content
    content = file.read()
    # Выводим содержимое файла на экран
    print(content)

Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\n\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\n\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\n\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случае каждый шаг является травмой для ногтевой пластины.\n\nТакже давление может оказываться на ногтевую пластину 

In [9]:
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=102, chunk_overlap=0)

for chunk in splitter.split_text(content):
    source_chunks.append(Document(page_content=chunk, metadata={}))

In [10]:
# всего получилось чанков:
len(source_chunks)

38

In [11]:
# первый чанк
print(source_chunks[0])
page_content = source_chunks[0].page_content
# длина первого чанка
print(len(page_content))

page_content='Онихолизис\\n\\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\\n\\nПри данной проблеме' metadata={}
100


In [12]:
# Устанавливаем метаданные для первого чанка
source_chunks[0].metadata = {"teg": "раздел 1"}

# выводим на печать обновленные метаданные чанка
print(source_chunks[0].metadata)

{'teg': 'раздел 1'}


In [14]:
loader = TextLoader("C:/Users/lenovo/Desktop/ITE Group/AI_STAFF/Neuro/BASE.txt", encoding='utf-8')

data_from_file = loader.load()

In [15]:
data_from_file[0]

Document(page_content='Онихолизис\\n\\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\\n\\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\\n\\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\\n\\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\\n\\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\\n\\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случае каждый шаг является травмой для ногтевой пластины.\\n\\nТакже давление мо

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(content):
    source_chunks.append(Document(page_content=chunk, metadata={}))
len(source_chunks)

4

In [18]:
source_chunks[2]

Document(page_content='ногтя, он становится желтым, серым\\n\\nТекстура становится неравномерной, появляются неровности\\n\\nНаблюдаются болевые ощущения при нажатии\\n\\nКожа вокруг пластины краснеет\\n\\nПризнаки аналогичны тем, что проявляются и при грибке. Но имеются определенные отличия.\\n\\nОнхолизис не является заразным, при контактах не передается. Отслоение пластины наблюдается на начальных этапах, тогда как при грибке этот процесс начинается в запущенной форме.\\n\\nСледует учесть, что онхолизис и грибок могут развиваться одновременно. Поэтому при появлении первых симптомов необходимо обратиться к подологу, дерматологу, микологу.\\n\\nОбработка ногтей при онихолизисе\\n\\nПервое, что нужно сделать – сдать анализы. Если есть инфекция, то только вылечив ее мы можем надеяться на полное восстановление ногтя.\\n\\nВторое – срезать отслоившуюся часть ногтя, провести необходимую зачистку, но только если клиент готов заниматься  восстановлением ногтевой пластины. Если этого не сдела

In [19]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()



In [20]:
import openai

In [21]:
# Создадим индексную базу из разделенных фрагментов текста
try:
    
    db = FAISS.from_documents(source_chunks, embeddings)
    
except Exception as e:
    print("Произошла ошибка:", e)

In [22]:
from openai import error

In [23]:
#pip install openai --upgrade

In [24]:
# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [25]:
query = "Как лечить онихолизис?"
docs = db.similarity_search(query, k=4)

print("**отрезок №1**", docs[0].page_content)
print("**отрезок №2**", docs[1].page_content)
print("**отрезок №3**", docs[2].page_content)
print("**отрезок №4**", docs[3].page_content)

**отрезок №1** Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\n\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\n\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\n\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случае каждый шаг является травмой для ногтевой пластины.\n\nТакже давление может оказываться на ног

In [26]:
docs_and_scores = db.similarity_search_with_score(query, k=4)

In [27]:
# первый фрагмент
print("Контент: ", docs_and_scores[0][0].page_content)
print("Оценка расстояния: ", docs_and_scores[0][1])

Контент:  Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\n\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\n\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\n\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случае каждый шаг является травмой для ногтевой пластины.\n\nТакже давление может оказываться на ногтевую

In [28]:
# второй фрагмент
print("Контент: ", docs_and_scores[1][0].page_content)
print("Оценка расстояния: ", docs_and_scores[1][1])

Контент:  После травм на дистальных фалангах может появляться экзостос (костно-хрящевое образование), которое постепенно увеличивается в размерах и начинает давить изнутри на ногтевую пластину, и мы можем видеть внешнее проявление в виде онихолизиса.\n\nТравма гипонихия. Это часто случается, когда клиенты слишком усердно чистят под свободным краем ногтевой пластины. Слишком длинный свободный край ногтевой пластины на пальцах ног.\n\nВызвать негативные процессы в структуре ногтя может некачественный лак, контакт с активными химическими агентами.\n\nСамой распространенной причиной онихолизиса на руках — являются химический или температурный ожег ногтевого ложа при покрытии гель-лаком.\n\nОнихомикоз (грибок ногтевой пластины) и такие кожные заболевания, как псориаз, дерматит, экзема также могут быть причиной онихолизиса.\n\nСимптомы\n\nЕсть несколько симптомов, характерных для данной патологии. Интенсивность их проявления зависит от стадии. При онихолизисе:\n\nНогтевые пластины становятся

In [31]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для ответа клиенту в ответе. Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

In [33]:
system="Ты-консультант в центре подологии, ответь на вопрос клиента на основе документа с информацией. Не придумывай ничего от себя, отвечай максимально по документу. Не упоминай Документ с информацией для ответа клиенту. Клиент ничего не должен знать про Документ с информацией для ответа клиенту"

In [34]:
# вопрос пользователя
topic= "У меня отделился ноготь. Как лечить?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\n\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\n\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\n\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случа

' Отделение ногтя, или онихолизис, требует внимательного подхода к лечению. В первую очередь, необходимо сдать анализы, чтобы исключить наличие инфекции. Если инфекция  обнаружена, её нужно вылечить для полного восстановления ногтя. Далее, важно срезать отслоившуюся часть ногтя и провести необходимую зачистку. Это поможет предотвратить  дальнейшее развитие проблемы. В домашнем уходе рекомендуется использовать профилактические препараты для предупреждения инфицирования и регенерирующие препараты для  восстановления ногтевого ложа. Также полезно принимать витамины для волос и ногтей. Не забывайте посещать специалиста раз в месяц для контроля процесса восстановления  ногтевой пластины.'

In [35]:
# вопрос пользователя
topic= "У меня отделился ноготь от пальца. Как самостоятельно его вставить на место?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим недостатком.\n\nМежду тем разрушенная структура создает высокий риск развития грибковых, бактериальных инфекций. Заниматься восстановлением ногтя необходимо, чтобы исключить серьезные последствия.\n\nПричины появления онихолизиса В большинстве случаев он является следствием травмы. Ногтевая пластина может быть повреждена при защемлении пальца, ударе. Если говорить об этой проблеме на пальцах ног, то основная причина – деформация стопы, которая приводит к деформации пальцев и как следствие – онихолизису.\n\nПричиной может быть неправильная обувь: узкая, маленькая, давящая сверху на пальцы, высокие каблуки. В этом случа

' Отделение ногтя от ногтевого ложа, или онихолизис, требует особого внимания и ухода. Важно понимать, что самостоятельно "вставить" ноготь на место невозможно и это может  привести к осложнениям. Рекомендуется следующее: 1. Обратитесь к специалисту, чтобы определить причину отделения ногтя и получить профессиональную помощь. 2. Сдайте  анализы для исключения инфекций. 3. Используйте профилактические препараты для предупреждения инфицирования. 4. Применяйте регенерирующие препараты для восстановления  ногтевого ложа и витамины для волос и ногтей. 5. Посещайте специалиста раз в месяц для контроля процесса восстановления. Самостоятельные попытки вернуть ноготь на место  могут усугубить ситуацию и привести к инфекциям или другим осложнениям.'

In [36]:
# вопрос пользователя
topic= "Какие есть риски в том, чтобы не восстанавливать ноготь?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================более сухой и, соответственно, будет скручиваться, деформируя ногтевую пластину не давая прирасти ногтю к ногтевому ложу.\n\nТретье – даже если анализ не подтвердил наличие инфекции, клиенту все равно необходимо использовать в домашнем уходе профилактические препараты для предупреждения инфицирования.\n\nИ последнее – это регенерирующие препараты для восстановления ногтевого ложа местного применения и витамины для волос и ногтей.\n\nНу и конечно один раз в месяц клиенту нужно будет посещать специалиста, который будет следить за процессом восстановления ногтевой пластины\n\n  
Отрывок документа №2
=====================Онихолизис\n\nОнихолизис – это отделение ногтевой пластины от ногтевого ложа.\n\nПри данной проблеме изменяется цвет пластины, ее структура, форма. В результате она отслаивается от ложа.\n\nМногие не уделяют онихолизису должного внимания, считают его косметическим 

' Если не заниматься восстановлением ногтя, существует несколько рисков. Во-первых, разрушенная структура ногтя создает высокий риск развития грибковых и бактериальных  инфекций. Во-вторых, ногтевая пластина может деформироваться, что приведет к изменению ее цвета, структуры и формы. В-третьих, могут возникнуть болевые ощущения и  покраснение кожи вокруг ногтя. Важно помнить, что онихолизис и грибок могут развиваться одновременно, поэтому при первых симптомах необходимо обратиться к специалисту.'